# Intracellular Electrophysiology: SweepTable 

## Step 1: Import the required packages

In [1]:
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile
import numpy as np
from pynwb.icephys import VoltageClampStimulusSeries
from pynwb.icephys import VoltageClampSeries

## Step 2: Create an ```NWBFile``` and ```Device```

In [2]:
nwbfile = NWBFile('my first synthetic recording', 'EXAMPLE_ID', datetime.now(tzlocal()),
                  experimenter='Dr. Bilbo Baggins',
                  lab='Bag End Laboratory',
                  institution='University of Middle Earth at the Shire',
                  experiment_description='I went on an adventure with thirteen dwarves to reclaim vast treasures.',
                  session_id='LONELYMTN')

In [3]:
device = nwbfile.create_device(name='Heka ITC-1600')

## Step 3: Create the electrodes

In [4]:
elec0 = nwbfile.create_icephys_electrode(name="elec0",
                                         description='a mock intracellular electrode',
                                         device=device)
elec1 = nwbfile.create_icephys_electrode(name="elec1",
                                        description='a mock intracellular electrode',
                                        device=device)

## Step 4: Create a stimulus/response recording

In [5]:
vcss0 = VoltageClampStimulusSeries(
    name="vcss0", data=[2, 3, 4, 5, 6], starting_time=234.5, rate=10e3, electrode=elec0, gain=0.03, sweep_number=0)
nwbfile.add_stimulus(vcss0)
vcs0 = VoltageClampSeries(
    name="vcs0", data=[0.1, 0.2, 0.3, 0.4, 0.5],
    conversion=1e-12, resolution=np.nan, starting_time=234.5, rate=20e3,
    electrode=elec0, gain=0.02, capacitance_slow=100e-12, resistance_comp_correction=70.0,
    sweep_number=0)
nwbfile.add_acquisition(vcs0)

PyNWB automatically populates the ```SweepTable``` for you.

In [6]:
nwbfile.sweep_table.to_dataframe()

,series,sweep_number
id,,
0,[vcss0 pynwb.icephys.VoltageClampStimulusSerie...,0
1,[vcs0 pynwb.icephys.VoltageClampSeries at 0x47...,0


Lets say we recorded from 2 electrodes symultaneously. In this case we have multiple PatchClampSeries with the same sweep number but different ```IntracellularElectrode``` obects. 

In [7]:
vcss1 = VoltageClampStimulusSeries(
    name="vcss1", data=[1, 2, 3, 3, 8], starting_time=264.5, rate=10e3, electrode=elec1, gain=0.03, sweep_number=0)
nwbfile.add_stimulus(vcss1)
vcs1 = VoltageClampSeries(
    name="vcs1", data=[0.15, 0.25, 0.35, 0.45, 0.55],
    conversion=1e-12, resolution=np.nan, starting_time=264.5, rate=20e3,
    electrode=elec1, gain=0.02, capacitance_slow=100e-12, resistance_comp_correction=70.0,
    sweep_number=0)
nwbfile.add_acquisition(vcs1)

In [8]:
nwbfile.sweep_table.to_dataframe()

,series,sweep_number
id,,
0,[vcss0 pynwb.icephys.VoltageClampStimulusSerie...,0
1,[vcs0 pynwb.icephys.VoltageClampSeries at 0x47...,0
2,[vcss1 pynwb.icephys.VoltageClampStimulusSerie...,0
3,[vcs1 pynwb.icephys.VoltageClampSeries at 0x47...,0


As we keep acquiring data we can now add data from additional sweeps, e.g:

In [9]:
sweep_number = 1
nwbfile.add_stimulus(VoltageClampStimulusSeries(
    name="vcss"+str(sweep_number+1), data=[2, 3, 4, 5, 6], 
    starting_time=234.5, rate=10e3, 
    electrode=elec1, gain=0.03, 
    sweep_number=sweep_number))
nwbfile.add_acquisition(VoltageClampSeries(
    name="vcs"+str(sweep_number+1), data=[0.1, 0.2, 0.3, 0.4, 0.5],
    conversion=1e-12, resolution=np.nan, starting_time=234.5, rate=20e3,
    electrode=elec1, gain=0.02, capacitance_slow=100e-12, resistance_comp_correction=70.0,
    sweep_number=sweep_number))

In [10]:
nwbfile.sweep_table.to_dataframe()

,series,sweep_number
id,,
0,[vcss0 pynwb.icephys.VoltageClampStimulusSerie...,0
1,[vcs0 pynwb.icephys.VoltageClampSeries at 0x47...,0
2,[vcss1 pynwb.icephys.VoltageClampStimulusSerie...,0
3,[vcs1 pynwb.icephys.VoltageClampSeries at 0x47...,0
4,[vcss2 pynwb.icephys.VoltageClampStimulusSerie...,1
5,[vcs2 pynwb.icephys.VoltageClampSeries at 0x47...,1


Since ```SweepTable``` is a ```DynamicTable``` we can easily add custom metadata about our experiments as additional columns.

In [11]:
nwbfile.sweep_table.add_column(name='notes', description='lab internal notes about the recordings',
                               data=['a', 'a', 'b', 'b', 'c', 'c'])

In [12]:
nwbfile.sweep_table.to_dataframe()

,series,sweep_number,notes
id,,,
0,[vcss0 pynwb.icephys.VoltageClampStimulusSerie...,0,a
1,[vcs0 pynwb.icephys.VoltageClampSeries at 0x47...,0,a
2,[vcss1 pynwb.icephys.VoltageClampStimulusSerie...,0,b
3,[vcs1 pynwb.icephys.VoltageClampSeries at 0x47...,0,b
4,[vcss2 pynwb.icephys.VoltageClampStimulusSerie...,1,c
5,[vcs2 pynwb.icephys.VoltageClampSeries at 0x47...,1,c


# Step 5: Querying the ```SweepTable```

Get all ```PatchClampSeries``` associated with a particular sweep:

In [13]:
sweep1 = nwbfile.sweep_table.get_series(1)
print([i.name for i in sweep1])

['vcss2', 'vcs2']


In [14]:
sweep0 = nwbfile.sweep_table.get_series(0)
print([i.name for i in sweep0])

['vcss0', 'vcs0', 'vcss1', 'vcs1']


For ``sweep_number=0`` we find that we have 4 simulaneously recorded ```PatchClampSeries```, i.e., 2 stimulus series and 2 response series. To identify which ones belong together we then need to look at the ```IntracellularElectrode``` that was used, i.e., corresponding series will use the same electrode.

In [15]:
sweep0 = nwbfile.sweep_table.get_series(0)
sweep0_pairs = {}
for s in sweep0:
    curr_pair = sweep0_pairs.get(s.electrode.name, None)
    if curr_pair:
        curr_pair.append(s)
    else:
        sweep0_pairs[s.electrode.name] = [s]

In [16]:
for k, v, in sweep0_pairs.items():
    print(k, ':' , [i.name for i in v])

elec0 : ['vcss0', 'vcs0']
elec1 : ['vcss1', 'vcs1']
